In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from IPython.display import display
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [22]:
df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
df.head(4)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
training_data = torch.tensor(df.values)
labels_data = torch.tensor(df["label"])


In [24]:
X_train, X_val, y_train, y_val = train_test_split(training_data, labels_data, test_size=0.2)

In [25]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)


torch.Size([33600, 785]) torch.Size([33600])
torch.Size([8400, 785]) torch.Size([8400])


In [26]:
train_loader = DataLoader(X_train, batch_size = 32, shuffle = True)
val_loader = DataLoader(X_val, batch_size = 32, shuffle = True)

In [27]:
USE_GPU = True
dtype = torch.float32
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [28]:
model = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 5, bias = True),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            
            nn.Conv2d(32, 64, kernel_size = 3, bias = True),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 1),    
            
            nn.Flatten(),
            nn.Linear(64 * 9 * 9, 128),
            nn.ReLU(),
    
            nn.Linear(128, 10),
            
                          )

In [31]:
epochs = 30
optimizer = optim.Adam(model.parameters(), lr=2e-4)
dtype = torch.float32

In [32]:
model = model.to(device=device)
PATH = "/kaggle/working/best_model.pth"
best_accuracy = 0

for e in range(epochs):
    model.train()
    training_accuracy = 0
    validation_accuracy = 0  
    loss_list = []
    denom_train = 0
    denom_val = 0
    print("EPOCH: ", e + 1)
    for (t, data) in enumerate(train_loader):
        optimizer.zero_grad()
        
        input_array = data[:,1:]/255.0        
        target = data[:,0]
        
        input_array = input_array.reshape(len(target),28,28).unsqueeze(1)
        
        input_array = input_array.to(device=device, dtype = dtype)  
        target = target.to(device=device)
        
        scores = model(input_array)
        predict = nn.LogSoftmax()
        predictions = predict(scores)
        predictions = torch.argmax(predictions, axis = 1)
        
        
        loss = nn.functional.cross_entropy(scores, target)
        loss.backward()

        optimizer.step()
        
        training_accuracy += sum(target == predictions).item()
        denom_train += target.shape[0]
        loss_list.append(loss.item())
    
    print("Got {}/{} correct".format(training_accuracy, denom_train))
    print("Loss = {:.4f}, Epoch Training Accuracy = {:.4f}\n".format(sum(loss_list) / len(loss_list), 
                                                                   ((training_accuracy / denom_train) * 100)))
    for (t_val, data_val) in enumerate(val_loader):    
        input_array_val = data_val[:,1:]/255.0        
        target_val = data_val[:,0]
        input_array_val = input_array_val.reshape(len(target_val),28,28).unsqueeze(1)
        
        input_array_val = input_array_val.to(device=device, dtype = dtype)  
        target_val = target_val.to(device=device)
        
        
        scores_val = model(input_array_val)
        predict_val = nn.LogSoftmax()
        predictions_val = predict_val(scores_val)
        predictions_val = torch.argmax(predictions_val, axis = 1)
        
        validation_accuracy += sum(target_val == predictions_val).item()
        denom_val += target_val.shape[0]
        
    
    print("Got {}/{} correct".format(validation_accuracy, denom_val))
    print("Epoch Validation Accuracy = {:.4f}\n".format((validation_accuracy / denom_val) * 100))
    
    if (validation_accuracy / denom_val) * 100 > best_accuracy:
        print("Saving the best model.......")
        best_accuracy = (validation_accuracy / denom_val) * 100
        torch.save(model, PATH)

print("Training Over")

        


EPOCH:  1


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Got 33453/33600 correct
Loss = 0.0125, Epoch Training Accuracy = 99.5625



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Got 8279/8400 correct
Epoch Validation Accuracy = 98.5595

Saving the best model.......
EPOCH:  2
Got 33494/33600 correct
Loss = 0.0086, Epoch Training Accuracy = 99.6845

Got 8282/8400 correct
Epoch Validation Accuracy = 98.5952

Saving the best model.......
EPOCH:  3
Got 33478/33600 correct
Loss = 0.0099, Epoch Training Accuracy = 99.6369

Got 8305/8400 correct
Epoch Validation Accuracy = 98.8690

Saving the best model.......
EPOCH:  4
Got 33522/33600 correct
Loss = 0.0075, Epoch Training Accuracy = 99.7679

Got 8286/8400 correct
Epoch Validation Accuracy = 98.6429

EPOCH:  5
Got 33494/33600 correct
Loss = 0.0089, Epoch Training Accuracy = 99.6845

Got 8295/8400 correct
Epoch Validation Accuracy = 98.7500

EPOCH:  6
Got 33494/33600 correct
Loss = 0.0119, Epoch Training Accuracy = 99.6845

Got 8302/8400 correct
Epoch Validation Accuracy = 98.8333

EPOCH:  7
Got 33544/33600 correct
Loss = 0.0061, Epoch Training Accuracy = 99.8333

Got 8314/8400 correct
Epoch Validation Accuracy = 98.97

In [33]:
df_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
df_test.head(4)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
test_data = torch.tensor(df_test.values)
print(test_data.shape)

torch.Size([28000, 784])


In [35]:
model = torch.load(PATH)

output_list = []
ids_list = []
# model.eval()
num_samples = test_data.shape[0]

for idx in range(num_samples):        
    
    input_array_test = (test_data[idx,:].unsqueeze(0))/255.0
    input_array_test = input_array_test.reshape(28,28).unsqueeze(0).unsqueeze(1)
    input_array_test = input_array_test.to(device=device, dtype = dtype)  
        
    scores_test = model(input_array_test)
    predict_test = nn.LogSoftmax()
    predictions_test = predict_test(scores_test)
    predictions_test = torch.argmax(predictions_test, axis = 1)
    
#     image = test_data[idx,:].reshape(28,28)
#     plt.imshow(image)
#     plt.xlabel(predictions_test.item(), fontsize = 20)
#     plt.show()
    
    ids_list.append(idx + 1)
    output_list.append(predictions_test.item())
        

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [36]:
print(output_list[:10])

[2, 0, 9, 0, 3, 7, 0, 3, 0, 3]


In [37]:
df_submission = pd.DataFrame({"ImageId":ids_list, "Label":output_list})
display(df_submission.head(10))

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [38]:
df_submission.to_csv("predictions.csv", index = False)